In [70]:
import pandas as pd
from geopy.distance import vincenty
import geohash
import numpy as np
import csv
import operator
import collections
import datetime
from datetime import datetime
from collections import defaultdict
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle

In [2]:
df = pd.read_csv('masterfile.csv')
df['ts'] = pd.to_datetime(df.time)

In [3]:
df.sort(columns=['ID'], inplace=True)
names=df['ID'].unique().tolist()

people = {}
for n in names:
    select = df.loc[df['ID']==n]
    people[n] = len(select['Day'].unique())

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [4]:
def round_to_5min(t):
    delta = pd.datetools.timedelta(minutes=t.minute%5, 
                               seconds=t.second, 
                               microseconds=t.microsecond)
    t -= delta
    if delta >= pd.datetools.timedelta(minutes=2.5):
        t += pd.datetools.timedelta(minutes=5)
    return t

In [5]:
df['ts10'] = df['ts'].apply(lambda x: round_to_5min(x))

In [6]:
drop_ids = [p for p in people if people[p] <= 5]
df = df.loc[~(df['ID'].isin(drop_ids))]

In [7]:
trial = df[['ts10','ID','lat','lon','accuracy']].drop_duplicates(subset=['ts10','ID'])

In [8]:
piv = pd.pivot_table(trial, index='ts10', columns='ID' ,values = ['lat','lon','accuracy'])

In [9]:
piv = piv.asfreq('5Min', method=None)

In [10]:
piv.groupby(pd.TimeGrouper('D')).count().tail()

lat                                                 \
ID         1rishabhtrivedi adithyapsv anshumanagrwl anuj7chauhan arch.b80   
ts10                                                                        
2016-04-30             288        181             0          189        4   
2016-05-01             253        285             0          237       16   
2016-05-02               0        208             0          208       22   
2016-05-03               0        277             0          189       26   
2016-05-04               0        205             0          250        2   

                                                                          \
ID         arkg1996 bhagyeshvikani bhardwaj.rish coolsush89 desai.deshna   
ts10                                                                       
2016-04-30        0            288           288        253            0   
2016-05-01        2            288           277        288          163   
2016-05-02        0            224            35        288          288   
2016-05-03        2            288           288        202          288   
2016-05-04        0            277           206        284          277   

             ...         accuracy                                             \
ID           ...    rudra.chandak sacheendra.t sagarparikh31 samriddhisimlai   
ts10         ...                                                               
2016-04-30   ...              285            0             1               0   
2016-05-01   ...              287            0             0               0   
2016-05-02   ...              280            0             0               0   
2016-05-03   ...              280            0             0               0   
2016-05-04   ...              282            0             0               0   

                                                                 \
ID         saumyadoshi shaleen.k.gupta tany.dudett umangjparmar   
ts10                                                              
2016-04-30         282               0         286          231   
2016-05-01         229               0         242          208   
2016-05-02         207               0          48          246   
2016-05-03         275               0         272          232   
2016-05-04         167               0         235          119   

                                     
ID         vaibhav29.07.97 vraj2026  
ts10                                 
2016-04-30             143      240  
2016-05-01             254      146  
2016-05-02             114      149  
2016-05-03               0      210  
2016-05-04               0       33  

[5 rows x 138 columns]

In [11]:
import itertools
ts = piv.index.tolist()

m = [list(g) for k, g in itertools.groupby(ts, key=lambda d: d.date())]
dates = list(set(map(lambda t: t.date(),ts)))

In [41]:
select = df.loc[df['ts10']=='2016-04-20 02:00:00']
print len(select)
print m[50][0]

116
2016-03-15 00:00:00


In [44]:
for i in range(50,len(m)):
    select = df.loc[df['ts10']=='2016-04-20 02:00:00']
    data_ = select[['lat','lon']]
    data = data_.as_matrix()
    kms_per_radian = 6371.0088
    epsilon = 0.02 / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(data))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.Series([data[cluster_labels == n] for n in range(num_clusters)])
    print('Number of clusters: {}'.format(num_clusters))
    break

Number of clusters: 5


In [69]:
print clusters[3]


[[ 23.190571  72.628358]
 [ 23.19051   72.628306]
 [ 23.19051   72.628268]
 [ 23.190514  72.62822 ]
 [ 23.190514  72.62822 ]
 [ 23.190474  72.628265]
 [ 23.190531  72.62819 ]
 [ 23.190564  72.628227]
 [ 23.190546  72.628195]
 [ 23.190534  72.628196]
 [ 23.190535  72.628198]
 [ 23.190537  72.628201]
 [ 23.190543  72.628197]
 [ 23.19053   72.628188]]


In [66]:
fig, ax = plt.subplots(figsize=[10, 6])
full_scatter = ax.scatter(data_['lat'], data_['lon'], c='y')
colors=['b','r','g','c','m']
for i in range(0,len(clusters)):
    lat_list = clusters[i][:,0]
    lon_list = clusters[i][:,1]
    ax.scatter(lat_list,lon_list, c=colors[i])
    np.savetxt(str(i)+'_cluster.csv',clusters[i])
    
ax.set_title('Full data set plus DBSCAN set')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()

In [64]:
data_.to_csv('FullData.csv')